In [3]:
from gurobi import *

In [4]:
rack_name, slots = multidict({
   'alpha' : 48
})

In [5]:
chips, quantity, powerdraw, slots_required = multidict({
    '1080' : [10, 45, 2],
    '2080' : [10, 55, 2]
})

In [6]:
m = Model("Tactus")

Academic license - for non-commercial use only


In [7]:
# always need the load on each phase Lead
L = m.addVars(3, name='phases')

In [8]:
# add x_ij where i is the chip and j is the possible orientations
# assume only 2 slots necessary
# orientation
# 1 = 1,2
# 2 = 1,3
# 3 = 2,3
# again the set is not an SOS
indicies = [(chip,orientation) for chip in chips for orientation in range(3)] # gen pairs 
x = m.addVars(indicies,vtype=GRB.INTEGER)

In [9]:
m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,1]) for chip in chips),GRB.EQUAL,L[0]);
m.addConstr(quicksum(powerdraw[chip]*(x[chip,0]+x[chip,2]) for chip in chips),GRB.EQUAL,L[1]);
m.addConstr(quicksum(powerdraw[chip]*(x[chip,1]+x[chip,2]) for chip in chips),GRB.EQUAL,L[2]);

In [10]:
m.addConstrs(quicksum(x[chip,orient] for orient in range(3)) == quantity[chip] for chip in chips);

In [11]:
# simplifies later constraints
m.addConstr(L[0],GRB.LESS_EQUAL,L[1])
m.addConstr(L[1],GRB.LESS_EQUAL,L[2])

<gurobi.Constr *Awaiting Model Update*>

In [12]:
m.setObjective(L[2]-L[0])

In [13]:
m.optimize()

Optimize a model with 7 rows, 9 columns and 25 nonzeros
Variable types: 3 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 5 rows, 7 columns, 21 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)
Found heuristic solution: objective 900.0000000
Found heuristic solution: objective 450.0000000

Root relaxation: objective 1.421085e-13, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    5  450.00000    0.00000   100%     -    0s
H    0     0                      60.0000000    0.00000   100%     -    0s
H    0     0                      50.0000000    0.00000   100%     -    0s
H    0     0                      40.00

<bound method tupledict.values of {('1080', 0): <gurobi.Var C3 (value 0.0)>, ('1080', 1): <gurobi.Var C4 (value 5.0)>, ('1080', 2): <gurobi.Var C5 (value 5.0)>, ('2080', 0): <gurobi.Var C6 (value 6.0)>, ('2080', 1): <gurobi.Var C7 (value 2.0)>, ('2080', 2): <gurobi.Var C8 (value 2.0)>}>

In [29]:
L

{0: <gurobi.Var phases[0] (value 665.0)>,
 1: <gurobi.Var phases[1] (value 665.0)>,
 2: <gurobi.Var phases[2] (value 670.0)>}